<a href="https://colab.research.google.com/github/cagBRT/computer-vision/blob/master/2_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
! pip install git+https://github.com/huggingface/transformers.git
     

# Preprocessing Data
Before you can use your data in a model, the data needs to be processed into an acceptable format for the model. A model does not understand raw text, images or audio. These inputs need to be converted into numbers and assembled into tensors. In this tutorial, you will:

>Preprocess textual data with a tokenizer.<br>
Preprocess image or audio data with a feature extractor.<br>
Preprocess data for a multimodal task with a processor.<br>

# NLP

The main tool for processing textual data is a tokenizer. A tokenizer starts by splitting text into tokens according to a set of rules. The tokens are converted into numbers, which are used to build tensors as input to a model. Any additional inputs required by a model are also added by the tokenizer.

**If you plan on using a pretrained model, it's important to use the associated pretrained tokenizer.** This ensures the text is split the same way as the pretraining corpus, and uses the same corresponding tokens-to-index (usually referrred to as the vocab) during pretraining.

Get started quickly by loading a pretrained tokenizer with the AutoTokenizer class. This downloads the vocab used when a model is pretrained.



# Tokenize
Load a pretrained tokenizer with AutoTokenizer.from_pretrained():

**Tokenizing** (splitting strings in sub-word token strings), converting tokens strings to ids and back, and encoding/decoding (i.e., tokenizing and converting to integers).<br>
**Adding new tokens** to the vocabulary in a way that is independent of the underlying structure (BPE, SentencePiece…).<br>
**Managing special tokens** (like mask, beginning-of-sentence, etc.): adding them, assigning them to attributes in the tokenizer for easy access and making sure they are not split during tokenization.


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Then pass your sentence to the tokenizer:

Examples of tokens:<br>
>101 - beginning of text<br>
102 - end of text<br>
119 - period
Punctuation is included

In [ ]:
encoded_input1 = tokenizer(".I and am you are the")
print(encoded_input1)

In [ ]:
encoded_input2 = tokenizer("I am. You are not.")
print(encoded_input2)

In [ ]:
encoded_input = tokenizer("Do not meddle in the affairs of wizards, for they are subtle and quick to anger.")
print(encoded_input)

The tokenizer returns a dictionary with three important itmes:<br>

- input_ids are the indices corresponding to each token in the sentence.<br>
- attention_mask indicates whether a token should be attended to or not.<br>
- token_type_ids identifies which sequence a token belongs to when there is more than one sequence.<br>

You can decode the input_ids to return the original input:

In [ ]:
tokenizer.decode(encoded_input["input_ids"])

As you can see, the tokenizer added two special tokens - CLS and SEP (classifier and separator) - to the sentence. Not all models need special tokens, but if they do, the tokenizer will automatically add them for you.

If there are several sentences you want to process, pass the sentences as a list to the tokenizer:

In [ ]:
batch_sentences = [
    "But what about second breakfast?",
    "Don't think he knows about second breakfast, Pip.",
    "What about elevensies?",
]
encoded_inputs = tokenizer(batch_sentences)
print(encoded_inputs)


**Pad**<br>
This brings us to an important topic. When you process a batch of sentences, they aren't always the same length. <br>
**This is a problem because tensors, the input to the model, need to have a uniform shape.** <br>
Padding is a strategy for ensuring tensors are rectangular by adding a special padding token to sentences with fewer tokens.<br>
**Set the padding parameter to True to pad the shorter sequences in the batch to match the longest sequence:**

In [ ]:

batch_sentences = [
    "But what about second breakfast?",
    "Don't think he knows about second breakfast, Pip.",
    "What about elevensies?",
]
encoded_input = tokenizer(batch_sentences, padding=True)
print(encoded_input)

**Truncation**<br>
Sometimes a sequence may be too long for a model to handle. In this case, you will need to truncate the sequence to a shorter length.

**Set the truncation parameter to True to truncate a sequence to the maximum length accepted by the model:**

In [ ]:
batch_sentences = [
    "But what about second breakfast?",
    "Don't think he knows about second breakfast, Pip.",
    "What about elevensies?",
]
encoded_input = tokenizer(batch_sentences, padding=True, truncation=True)
print(encoded_input)

**Build tensors**<br>
Finally, you want the tokenizer to return the actual tensors that are fed to the model.

Set the return_tensors parameter to either pt for PyTorch, or tf for TensorFlow:



In [ ]:
batch_sentences = [
    "But what about second breakfast?",
    "Don't think he knows about second breakfast, Pip.",
    "What about elevensies?",
]
encoded_input = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="pt")
print(encoded_input)


In [ ]:
batch_sentences = [
    "But what about second breakfast?",
    "Don't think he knows about second breakfast, Pip.",
    "What about elevensies?",
]
encoded_input = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="tf")
print(encoded_input)


# Pad and truncate
Just like the tokenizer, you can apply padding or truncation to handle variable sequences in a batch. Take a look at the sequence length of these two audio samples:



In [ ]:
from datasets import load_dataset, Audio

dataset = load_dataset("PolyAI/minds14", name="en-US", split="train")

In [ ]:
dataset[0]["audio"]


In [ ]:
dataset[0]["audio"]["array"].shape


In [ ]:
dataset[1]["audio"]["array"].shape


As you can see, the first sample has a longer sequence than the second sample. Let's create a function that will preprocess the dataset. Specify a maximum sample length, and the feature extractor will either pad or truncate the sequences to match it:

In [ ]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

In [ ]:

def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=16000,
        padding=True,
        max_length=100000,
        truncation=True,
    )
    return inputs

Apply the function to the the first few examples in the dataset:



In [ ]:
processed_dataset = preprocess_function(dataset[:5])

Now take another look at the processed sample lengths:

In [ ]:

processed_dataset["input_values"][0].shape


In [ ]:
processed_dataset["input_values"][1].shape


The lengths of the first two samples now match the maximum length you specified.

# Vision
A feature extractor is also used to process images for vision tasks. Once again, the goal is to convert the raw image into a batch of tensors as input.

Let's load the food101 dataset for this tutorial. Use 🤗 Datasets split parameter to only load a small sample from the training split since the dataset is quite large:

In [ ]:
from datasets import load_dataset
dataset = load_dataset("food101", split="train[:100]")

Next, take a look at the image with 🤗 Datasets Image feature:

In [ ]:
dataset[0]["image"]

We can use convert to change the image. 

In [ ]:
img1=dataset[0]["image"].convert(mode="L")
img1

Feature extractor
Load the feature extractor with AutoFeatureExtractor.from_pretrained():

In [ ]:
from transformers import ViTImageProcessor

feature_extractor = AutoFeatureExtractor.from_pretrained("google/vit-base-patch16-224")

Data augmentation
For vision tasks, it is common to add some type of data augmentation to the images as a part of preprocessing. You can add augmentations with any library you'd like, but in this tutorial, you will use torchvision's transforms module.

Normalize the image and use Compose to chain some transforms - RandomResizedCrop and ColorJitter - together:

In [ ]:
from torchvision.transforms import Compose, Normalize, RandomResizedCrop, ColorJitter, ToTensor

normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
transformations_to_array = Compose([ToTensor(), normalize]
)

The model accepts pixel_values as it's input. This value is generated by the feature extractor. Create a function that generates pixel_values from the transforms:

In [ ]:
from PIL import Image


In [ ]:
def transforms(examples):
    examples["pixel_values"] = [transformations_to_array(image.convert("RGB")) for image in examples["image"]]
    return examples

In [ ]:
tr=transformations_to_array(dataset[0]["image"]) 

Then use 🤗 Datasets set_transform to apply the transforms on-the-fly:

In [ ]:
tr

In [ ]:
#dataset.set_transform(transformations_to_array)

Now when you access the image, you will notice the feature extractor has added the model input pixel_values:

Here is what the image looks like after you preprocess it. Just as you'd expect from the applied transforms, the image has been randomly cropped and it's color properties are different.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.imshow(tr.permute(1, 2, 0))


In [ ]:
dataset[1]["image"]

In [ ]:
transforms_to_resize = Compose(
    [RandomResizedCrop(feature_extractor.size), ColorJitter(brightness=0.5, hue=0.5), ToTensor(), normalize])

In [ ]:
tr_rs=transformations_to_array(dataset[1]["image"]) 

In [ ]:

plt.imshow(tr_rs.permute(1, 2, 0))


Multimodal
For multimodal tasks. you will use a combination of everything you've learned so far and apply your skills to a automatic speech recognition (ASR) task. This means you will need a:

Feature extractor to preprocess the audio data.
Tokenizer to process the text.
Let's return to the LJ Speech dataset:

In [ ]:
from datasets import load_dataset

lj_speech = load_dataset("lj_speech", split="train")

Since you are mainly interested in the audio and text column, remove the other columns:

In [ ]:
lj_speech = lj_speech.map(remove_columns=["file", "id", "normalized_text"])

Now take a look at the audio and text columns:



In [ ]:
lj_speech[0]["audio"]

In [ ]:

lj_speech[0]["text"]

Remember from the earlier section on processing audio data, you should always resample your audio data's sampling rate to match the sampling rate of the dataset used to pretrain a model:

In [ ]:
lj_speech = lj_speech.cast_column("audio", Audio(sampling_rate=16_000))

Processor
A processor combines a feature extractor and tokenizer. Load a processor with [`AutoProcessor.from_pretrained]:

In [ ]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")

Create a function to process the audio data to input_values, and tokenizes the text to labels. These are your inputs to the model:


In [ ]:
def prepare_dataset(example):
    audio = example["audio"]

    example["input_values"] = processor(audio["array"], sampling_rate=16000)

    with processor.as_target_processor():
        example["labels"] = processor(example["text"]).input_ids
    return example
     

Apply the prepare_dataset function to a sample:

In [ ]:
prepare_dataset(lj_speech[0])


Notice the processor has added input_values and labels. The sampling rate has also been correctly downsampled to 16kHz.

Awesome, you should now be able to preprocess data for any modality and even combine different modalities! In the next tutorial, learn how to fine-tune a model on your newly preprocessed data.